<a href="https://colab.research.google.com/github/lucaschenu/Archivo-materia-IA/blob/main/WEEKLY_REPORT_3_LUCAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importações principais
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# CARREGAR OS DADOS
from google.colab import files
uploaded = files.upload()

Saving 01-04T184148_000_mode1.csv to 01-04T184148_000_mode1 (1).csv


In [ ]:
data = pd.read_csv("01-04T184148_000_mode1.csv")

In [ ]:
# Coluna de saída
target_col = "pSpintor::VAX_speed"

X = data.drop(columns=[target_col])
y = data[target_col]

In [ ]:
# DIVISÃO TREINO/TESTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Padronização (necessária para SVR, Lasso e Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# TREINAR OS MODELOS
models = {
    "OLS": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.001),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "SVR": SVR(kernel="rbf", C=100, epsilon=0.1),
    "XGBoost": XGBRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
}
results = []

for name, model in models.items():
    if name in ["SVR", "Lasso", "Ridge"]:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append([name, rmse, mae, r2])

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.626e+06, tolerance: 9.097e+02
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# RESULTADOS EM TABELA

results_df = pd.DataFrame(results, columns=["Modelo", "RMSE", "MAE", "R²"])
results_df = results_df.sort_values(by="R²", ascending=False)
print(results_df)

          Modelo       RMSE        MAE        R²
3  Random Forest   0.416761   0.051881  0.999966
5        XGBoost   1.297475   0.159527  0.999669
4            SVR  14.096890   2.767754  0.960922
0            OLS  42.256139  34.640957  0.648869
2          Lasso  43.344343  36.564747  0.630551
1          Ridge  43.672026  37.470296  0.624944
